In [ ]:
from scapy.all import *
import socket

## Client & server UDP simulation

- they must run on diffrent thread/process

#### -Server UDP

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(("0.0.0.0", 12345)) # 0.0.0.0 to listen to all Network card interfaces

print("UDP server waiting for messages...")

while True:
    data, address = server_socket.recvfrom(1024) # wait and listen until 1024 bytes received
    print(f'Message received : {data.decode()} de {address}') # address = (#port , #ip)
    response = f'Message received : {data.decode()}'
    server_socket.sendto(response.encode(), address)

UDP server waiting for messages...


#### -Client UDP

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ("127.0.0.1", 12345)

while True:
 message = input("Enter a message to send: ")
 client_socket.sendto(message.encode(), server_address)
 response, _ = client_socket.recvfrom(1024)
 print(f'Server reponse : {response.decode()}')

## Generate UDP segment

-they should run on diffrent thread/process

#### - sending udp

In [10]:
packet = IP(dst="10.108.100.8") / UDP(dport=12345) / "Hello UDP" # update the IP adress
print('Packet generated :')
packet.show()

send(packet)

Packet generated :
###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = udp
  chksum    = None
  src       = 10.225.62.193
  dst       = 10.108.100.8
  \options   \
###[ UDP ]### 
     sport     = domain
     dport     = 12345
     len       = None
     chksum    = None
###[ Raw ]### 
        load      = 'Hello UDP'


Sent 1 packets.


#### -receive udp

In [ ]:
print("Waiting for UDP reponse...")
sniff(filter="udp and port 12345", prn=lambda x: x.show(), count=5)

#### -generating an incorrect checksum

In [ ]:
packet = IP(dst='10.180.100.8') / UDP(dport=12345, sport=54321, chksum=0xFFFF) / "Invalid checksum" # update IP address
print('Packet with incorecct checksum:')
packet.show()

send(packet)

Packet with incorecct checksum:
###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = udp
  chksum    = None
  src       = 10.225.62.193
  dst       = 10.180.100.8
  \options   \
###[ UDP ]### 
     sport     = 54321
     dport     = 12345
     len       = None
     chksum    = 0xffff
###[ Raw ]### 
        load      = 'Invalid checksum'



## UDP scanner simulation

In [ ]:
target_ip = "192.168.110.15"
ports = [53, 67, 123, 500] # commonly used UDP ports

for port in ports:
    packet = IP(dst=target_ip) / UDP(dport=port)
    reponse = sr1(packet, timeout=1, verbose=0)
    if response:
        print(f'Respoonse received on port {port}')
    else:
        print(f'No reponse on port {port}')

## UDP for DNS simulation

- they should run on diffrent thread/process

#### - server

In [ ]:
def handle_requedt(packet):
    if UDP in packet and packet[UDP].dport== 5353:
        print(f'Request received : {packet[Raw].load.decode()}')
        # creating the UDP answer
        reponse = IP(dst=packet[IP].src, src=packet[IP].dst) / UDP(dport=packet[UDP].sport, sport=5353) / f"186.168.1.105"
        print('Response sent: 192.168.1.105')
        send(response)

print('Server waiting...')
sniff(filter="udp and port 5353", prn=handle_requedt)

#### -client

In [ ]:
server_ip = "127.0.0.1"
server_port = 5353

domain_name = "www.example.com"

packet = IP(dst=server_ip) / UDP(dport=server_port, sport=12345) / domain_name
print('Request sent:')
packet.show()

reponse = sr1(packet, timeout=2)
if reponse:
    print(f'Answer received: {reponse[Raw].load.decode()}')
else:
    print('No reply received.')